In [1]:
import pandas
import numpy
import datetime

In [2]:
def convert_to_datetime(row, datecolName="dateStr", timeColName="timeStr"):
    currDateStr = str( int( row[datecolName] ) )
#     return currDateStr
    if row[timeColName] < 10:
        currTimeStr = "000" + str( int( row[timeColName] ) )
    elif row[timeColName] < 100:
        currTimeStr = "00" + str( int( row[timeColName] ) )
    elif row[timeColName] < 1000:
        currTimeStr = "0" + str( int( row[timeColName] ) )
    else:
        currTimeStr = str( int( row[timeColName] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

In [3]:
# Read SAPS data file!
datFileName = "../data/processedSaps-rbsp.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ')
# add dst_bins
dstBins = [ -150, -75, -50, -25, -10, 10 ]
sapsDataDF = pandas.concat( [ sapsDataDF, \
                    pandas.cut( sapsDataDF["dst_index"], \
                               bins=dstBins ) ], axis=1 )
sapsDataDF.columns = [ ["dateStr", "hour","sapsLat", \
                     "sapsMLT", "sapsVel", "radId", "poesLat",\
                     "poesMLT", "dst_date", "dst_index", "timeStr", "dst_bin"] ]
sapsDataDF = sapsDataDF.drop(["radId", "poesLat", "poesMLT", "dst_date", "hour"], 1)
# Round off MLT to nearest half
sapsDataDF["sapsMLTRounded"] = [ round(x * 2) / 2 \
                                for x in sapsDataDF["sapsMLT"] ]
# Add a date column
sapsDataDF['date']= sapsDataDF.apply( convert_to_datetime, axis=1 )
sapsDataDF['dateStr']= sapsDataDF['date'].apply( lambda x: x.strftime('%Y%m%d') )
sapsDataDF.head()

,dateStr,sapsLat,sapsMLT,sapsVel,dst_index,timeStr,dst_bin,sapsMLTRounded,date
0,20110107,56.5,17.7543,308.2077,-18.0,0,"(-25, -10]",18.0,2011-01-07
1,20110107,55.5,18.0147,224.1588,-18.0,0,"(-25, -10]",18.0,2011-01-07
2,20110107,56.5,17.8749,307.4328,-18.0,0,"(-25, -10]",18.0,2011-01-07
3,20110107,55.5,18.1324,222.4787,-18.0,0,"(-25, -10]",18.0,2011-01-07
4,20110107,56.5,17.9955,305.4201,-18.0,0,"(-25, -10]",18.0,2011-01-07


In [4]:
def convert_excel_sheet_datetime(row):
    if row["mm"] < 10:
        currMntStr = "0" + str(row["mm"])
    else:
        currMntStr = str(row["mm"])
    if row["dd"] < 10:
        currDtStr = "0" + str(row["dd"])
    else:
        currDtStr = str(row["dd"])
        
    currDateStr = "20" + str(row["yy"]) + currMntStr + currDtStr
    return datetime.datetime.strptime( currDateStr, "%Y%m%d" )

intDatesVAPDF = pandas.read_excel( "../data/days_interesting_SAPS.xlsx" )
intDatesVAPDF['date']= intDatesVAPDF.apply( convert_excel_sheet_datetime, axis=1 )
intDatesVAPDF['dateStr']= intDatesVAPDF['date'].apply( lambda x: x.strftime('%Y%m%d') )
intDatesVAPDF.head()

,yy,mm,dd,date,dateStr
0,12,12,2,2012-12-02,20121202
1,12,12,15,2012-12-15,20121215
2,12,12,17,2012-12-17,20121217
3,12,12,18,2012-12-18,20121218
4,12,12,20,2012-12-20,20121220


In [5]:
commonDatesDF = pandas.merge( sapsDataDF, intDatesVAPDF, on=['dateStr'] )
commonDatesDF.head()

,dateStr,sapsLat,sapsMLT,sapsVel,dst_index,timeStr,dst_bin,sapsMLTRounded,date_x,yy,mm,dd,date_y
0,20121202,64.5,21.2185,284.0258,-8.0,130,"(-10, 10]",21.0,2012-12-02 01:30:00,12,12,2,2012-12-02
1,20121202,64.5,21.3733,266.5380,-8.0,130,"(-10, 10]",21.5,2012-12-02 01:30:00,12,12,2,2012-12-02
2,20121202,64.5,21.5282,247.9277,-8.0,130,"(-10, 10]",21.5,2012-12-02 01:30:00,12,12,2,2012-12-02
3,20121202,64.5,21.6830,228.3907,-8.0,130,"(-10, 10]",21.5,2012-12-02 01:30:00,12,12,2,2012-12-02
4,20121202,64.5,21.8378,208.1250,-8.0,130,"(-10, 10]",22.0,2012-12-02 01:30:00,12,12,2,2012-12-02


In [6]:
intDatesVAPDF.shape
print commonDatesDF["dateStr"].unique().tolist()

['20121202', '20130120', '20130222', '20130301', '20130316', '20130424', '20130514', '20130624', '20130710', '20130821', '20130822', '20130823', '20140608', '20140617', '20140618', '20140619', '20140703', '20140710', '20140712', '20140726', '20140728', '20140802', '20140804', '20140805', '20140808', '20140911', '20141001', '20141011', '20141017', '20141018', '20141115', '20141116']


In [10]:
commonDatesArray = commonDatesDF["dateStr"].unique()
numpy.savetxt( "../data/inj-commondates.csv", commonDatesArray, fmt='%s')
# commonDatesDF.to_csv( "../data/inj-commondates.csv", header=None, columns=["dateStr"], index=None )